<a href="https://colab.research.google.com/github/twelvesense/first-repository/blob/master/ExplNode06.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 프로젝트: 

# 1 데이터 준비 

## 1-1. 필요한 라이브러리 불러오기

In [ ]:
import os, re 
import glob
import tensorflow as tf
# from tensorflow import keras
# import numpy as np

In [22]:
import warnings
warnings.filterwarnings("ignore")
import os, re
from PIL import Image
import glob
import numpy as np
from os.path import join
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split

## 1-2. corpus 파일 download하기

In [ ]:
!rm -r /content/lyrics
!mkdir -p /content/lyrics
!wget -O /content/lyrics.zip https://raw.githubusercontent.com/twelvesense/first-repository/master/data/lyrics/lyrics.zip
!unzip /content/lyrics.zip -d /content/lyrics
!rm /content/lyrics.zip

In [40]:
txt_file_path = '/content/lyrics/*'
txt_list = glob.glob(txt_file_path)

raw_corpus = []

# 여러개의 txt 파일을 모두 읽어서 raw_corpus 에 담습니다.
# for txt_file in txt_list:
# file = open('/content/lyrics/britney-spears.txt', 'r')
file = open('test.txt', 'w')
file.write('test only')
!ls
# raw = file.read().splitlines()
# with open('/content/lyrics/britney-spears.txt', 'r') as f:
    # raw = f.read().splitlines()
    # raw_corpus.extend(raw)
# for txt_file in txt_list:
    # print(txt_file)
    # with open(txt_file, "r") as f:
    #     raw = f.read().splitlines()
    #     raw_corpus.extend(raw)
# print("데이터 크기:", len(raw_corpus))
# print("Examples:\n", raw_corpus[:3])

TypeError: ignored